In [1]:
import os
import gc
import numpy as np
import random as rn
import pandas as pd
import tensorflow as tf
from keras import callbacks
from parameters import *

print(tf.config.list_physical_devices('GPU'))

tf.config.experimental.set_memory_growth(
    tf.config.list_physical_devices('GPU')[0], True)

rn.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import os
import requests
import numpy as np
import random as rn
import tensorflow as tf
from keras import layers, Sequential
from parameters import *

rn.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

input_0 = layers.Input(shape=(380, 380, 3))
input_0 = layers.RandomFlip(seed=SEED)(input_0)
input_0 = layers.RandomContrast(factor=0.1, seed=SEED)(input_0)
input_0 = layers.RandomRotation(factor=(-0.45, 0.45), seed=SEED)(input_0)

r = requests.get('http://storage.googleapis.com/tensorflow/keras-applications/efficientnet_v2/efficientnetv2-l_notop.h5', allow_redirects=True)
with open('model.h5', 'wb') as f:
    f.write(r.content)
model_path = os.path.join(MODEL_FOLDER_PATH, PRE_TRAINED_MODEL_NAME)

effnet = tf.keras.applications.efficientnet_v2.EfficientNetV2L(weights='model.h5',
                                                               include_top=False,
                                                               input_tensor=input_0, 
                                                               input_shape=(IMG_SIZE, IMG_SIZE, CHANNELS))

effnet.trainable = False

effnet_no_top_block = effnet.layers[-4].output

def build_top(effnet_output, suffix, output_num):
    conv = layers.Conv2D.from_config(effnet.layers[-3].get_config())
    conv._name = f'top_conv_{suffix}'
    top_component = conv(effnet_output)
    bn = layers.BatchNormalization.from_config(effnet.layers[-2].get_config())
    bn._name = f'top_bn_{suffix}'
    top_component = bn(top_component)
    act = layers.Activation.from_config(effnet.layers[-1].get_config())
    act._name = f'top_activation_{suffix}'
    top_component = act(top_component)
    
    top_component = layers.GlobalAveragePooling2D()(top_component)
    top_component = layers.BatchNormalization()(top_component)
    top_component = layers.Dropout(0.5)(top_component)
    
    top_component = layers.Dense(output_num, activation='softmax')(top_component)
    return top_component

top_output_1 = build_top(effnet_no_top_block, 1, 5)
top_output_2 = build_top(effnet_no_top_block, 2, 2)

top_block = tf.keras.Model(inputs=effnet.input, outputs=[top_output_1, top_output_2])

for i in [-3, -2, -1]:
    top_block.layers[2*i-8].set_weights(effnet.layers[i].get_weights())
    top_block.layers[2*i-7].set_weights(effnet.layers[i].get_weights())

top_block.build((None, IMG_SIZE, IMG_SIZE, CHANNELS))

In [ ]:
tf.keras.utils.plot_model(top_block, to_file='model.pdf', show_shapes=True, show_layer_activations=True, expand_nested=True)